In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

In [ ]:
import pydicom
import os
import numpy as np
import pandas as pd
from matplotlib import cm
from matplotlib import pyplot as plt
import cv2
import seaborn as sns
from tqdm import tqdm
import glob2

# Read IMGs

In [ ]:
#for training only
def show_dcm_info(dataset):
    print("Filename.........:", file_path)
    print("Storage type.....:", dataset.SOPClassUID)
    print()

    pat_name = dataset.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient's name......:", display_name)
    print("Patient id..........:", dataset.PatientID)
    print("Patient's Age.......:", dataset.PatientAge)
    print("Patient's Sex.......:", dataset.PatientSex)
    print("Modality............:", dataset.Modality)
    print("Body Part Examined..:", dataset.BodyPartExamined)
    print("View Position.......:", dataset.ViewPosition)
    
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print("Pixel spacing....:", dataset.PixelSpacing)

In [ ]:
def plot_pixel_array(dataset, figsize=(10,10)):
    plt.figure(figsize=figsize)
    plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)
    plt.show()

In [ ]:
for file_path in glob2.glob('../input/siim-train-test/dicom-images-train/**/*.dcm'):
    dataset = pydicom.dcmread(file_path)
    show_dcm_info(dataset)
    plot_pixel_array(dataset)
    print(dataset.pixel_array)
    break

In [ ]:
sz = 256
sz0 = 1024
PATH_TRAIN0 = '../input/siim-train-test/dicom-images-train/'
PATH_TRAIN1 = '../input/siim-train-test/dicom-images-test/'
PATH_TEST = '../input/siim-acr-pneumothorax-segmentation/stage_2_images/'
train_out = 'train.zip'
test_out = 'test.zip'
mask_out = 'masks.zip'
train = glob2.glob(os.path.join(PATH_TRAIN0, '**/*.dcm'))
test = glob2.glob(os.path.join(PATH_TEST, '**/*.dcm'))

In [ ]:
len(train)

In [ ]:
import pydicom
import skimage, os
import numpy as np
FolderPathName = r'../input/siim-train-test/dicom-images-train/'
slices = [pydicom.read_file(imagename) for imagename in glob2.glob(os.path.join(PATH_TRAIN0, '**/*.dcm'))]
#slices = [pydicom.data.get_testdata_file(FolderPathName + imagename) for imagename in os.listdir(FolderPathName)]
# Sort the dicom slices in their respective order
slices.sort(key=lambda x: int(x.InstanceNumber))
#jpg_folder = '../input/siim-train-test/dicom-images-jpg/' # Set your jpg folder
#for idx in range(len(slices)): 
    #write the images in a new folder
    #jpg_filepath = os.path.join( jpg_folder, "pic-{}.jpg".format(idx) )
    #np_pixel_array = slices[idx].pixel_array
    #cv2.imwrite(jpg_filepath, np_pixel_array)

In [ ]:
len(slices)

In [ ]:
np_pixel_array1 = []

In [ ]:
np_pixel_array2 = []
np_pixel_array3 = []

In [ ]:
for idx in range(len(slices[:4000])): 
    np_pixel_array1.append(slices[idx].pixel_array)

In [ ]:
len(np_pixel_array1[0])

In [ ]:
for idx in range(len(slices[4000:8000])): 
    np_pixel_array2.append(slices[idx].pixel_array)

In [ ]:
for idx in range(len(slices[8000:])): 
    np_pixel_array3.append(slices[idx].pixel_array)

In [ ]:
def normalize(pix_array):
    for i in range(len(pix_array)):
        pix_array[i] = pix_array[i]/np.sqrt(sum(sum(pix_array[i])))
    return pix_array

In [ ]:
for i in range(len(np_pixel_array1)):
    np_pixel_array1[i] = cv2.resize(np_pixel_array1[i], (256, 256))

In [ ]:
len(np_pixel_array1[0])

In [ ]:
for i in range(len(np_pixel_array2)):
    np_pixel_array2[i] = cv2.resize(np_pixel_array2[i], (256, 256))

In [ ]:
for i in range(len(np_pixel_array3)):
    np_pixel_array3[i] = cv2.resize(np_pixel_array3[i], (256, 256))

In [ ]:
np_pixels = np.concatenate((np_pixel_array1, np_pixel_array2, np_pixel_array3), axis=0)

In [ ]:
len(np_pixels)

# New approach

In [ ]:
from glob import glob
train_fns = sorted(glob('../input/siim-train-test-2/dicom-images-train/**/*.dcm'))
test_fns = sorted(glob('../input/siim-train-test-2/dicom-images-test/**/*.dcm'))

In [ ]:
train_fns = glob2.glob('../input/siim-train-test/dicom-images-train/**/*.dcm')
test_fns = glob2.glob('../input/siim-train-test/dicom-images-test/**/*.dcm')

In [ ]:
print(len(train_fns))
print(len(test_fns))

In [ ]:
from matplotlib import pyplot as plt
import pydicom

for k in range(10):
    fn = np.random.choice(test_fns)
    img = pydicom.read_file(fn).pixel_array
    plt.imshow(img, cmap='bone')
    plt.axis('off')
    plt.show()

In [ ]:
import pandas as pd
from collections import defaultdict
rles = pd.read_csv('../input/siim-train-test/train-rle.csv')
# images can have multiple annotations
rles_ = defaultdict(list)
for image_id, rle in zip(rles['ImageId'], rles[' EncodedPixels']):
    rles_[image_id].append(rle)
rles = rles_
annotated = {k: v for k, v in rles.items() if v[0] != '-1'}
print("%d of %d images are annotated" % (len(annotated), len(rles)))

In [ ]:
pixels = []
doubled = []
for fn in train_fns:
    img = cv2.resize(pydicom.read_file(fn).pixel_array, (256,256))
    pixels.append(img)

In [ ]:
len(pixels[0])

In [ ]:
len(pixels)

In [ ]:
df = pd.read_csv('../input/siim-train-test/train-rle.csv')

In [ ]:
train_id = []
for i in range(len(train)):
    train_id.append(train[i].split('/')[6].split('.dcm')[0])

In [ ]:
df = df.drop_duplicates('ImageId')

In [ ]:
df.info()

In [ ]:
df['ImageId'][0] #12047

In [ ]:
train_id[0] #12089

In [ ]:
missed = []
for i in range(len(df['ImageId'])):
    if df['ImageId'][i] in train_id:
        missed.append(i)
    else:
        continue

In [ ]:
df[' EncodedPixels']

In [ ]:
len(df[~df['ImageId'].isin(train_id)])

In [ ]:
contains = [df['ImageId'].str.contains(i) for i in train_id]

In [ ]:
np.any(df['ImageId']!= train_id[12048])

In [ ]:
np.any(train_id != df['ImageId'][12000])

In [ ]:
dicom_attr = ['AccessionNumber', 
              'BitsAllocated', 
              'BitsStored', 
              'BodyPartExamined', 
              'Columns', 
              'ConversionType', 
              'HighBit',
              'InstanceNumber',
              'LossyImageCompression',
              'LossyImageCompressionMethod',
              'Modality',
              'PatientAge',
              'PatientBirthDate',
              'PatientID',
              'PatientName',
              'PatientOrientation',
              'PatientSex',
              'PhotometricInterpretation',
#               'PixelData',
              'PixelRepresentation',
              'PixelSpacing',
              'ReferringPhysicianName',
              'Rows',
              'SOPClassUID',
              'SOPInstanceUID',
              'SamplesPerPixel',
              'SeriesDescription',
              'SeriesInstanceUID',
              'SeriesNumber',
              'SpecificCharacterSet',
              'StudyDate',
              'StudyID',
              'StudyInstanceUID',
              'StudyTime',
              'ViewPosition']

In [ ]:
from joblib import Parallel, delayed
import psutil
def create_features(fp):
    ret = []
    ret.append(fp.split('/')[-1][:-4])
    dataset = pydicom.dcmread(fp)
    for da in dicom_attr:
        ret.append(dataset.__getattr__(da))
    return np.array(ret).T


dicom_df = Parallel(n_jobs=psutil.cpu_count(), verbose=1)(
    (delayed(create_features)(fp) for fp in glob2.glob('../input/siim-train-test/dicom-images-train/**/*.dcm'))
)
dicom_df = pd.DataFrame(np.array(dicom_df), columns=['ImageId'] + dicom_attr)
dicom_df.head()

In [ ]:
useful_col = ['ImageId',
              'PatientAge', 
              'PatientSex', 
#               'ReferringPhysicianName',
              'ViewPosition', 
              'PixelSpacing']
df = df.merge(dicom_df.loc[:, useful_col], how='left', on='ImageId')
df.loc[:, 'IsDisease'] = (df.loc[:, ' EncodedPixels'] != ' -1')
df.head()

In [ ]:
my_dict = {i:train_id.count(i) for i in train_id}

In [ ]:
for i in range(len(df[' EncodedPixels'])):
    if df[' EncodedPixels'][i] == '-1':
        df[' EncodedPixels'][i] = 0
    else:
            df[' EncodedPixels'][i] = 1

In [ ]:
labels = df[' EncodedPixels']